# Store Item Demand Forecasting - Enhanced EDA

このノートブックでは、売上予測に必要な「データの勘所」を掴むために、詳細な可視化と分析を行います。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import os

%matplotlib inline
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (15, 8)

## 1. データの読み込み

In [ ]:
DATA_DIR = "/work/competitions/demand-forecasting-kernels-only/data"
train = pd.read_csv(f"{DATA_DIR}/train.csv", parse_dates=['date'])

train['year'] = train['date'].dt.year
train['month'] = train['date'].dt.month
train['dayofweek'] = train['date'].dt.dayofweek

print(f"Train shape: {train.shape}")
train.head()

## 2. 全体的な傾向と季節性

In [ ]:
# 2.1 年次トレンド
plt.figure(figsize=(12, 6))
sns.lineplot(x='year', y='sales', data=train, estimator='mean')
plt.title('Average Yearly Sales Trend')
plt.show()

In [ ]:
# 2.2 月次季節性 (Monthly Seasonality)
plt.figure(figsize=(12, 6))
sns.boxplot(x='month', y='sales', data=train)
plt.title('Monthly Sales Distribution')
plt.show()

In [ ]:
# 2.3 曜日別季節性 (Weekly Seasonality)
plt.figure(figsize=(12, 6))
sns.boxplot(x='dayofweek', y='sales', data=train)
plt.title('Weekly Sales Distribution (0=Monday, 6=Sunday)')
plt.show()

## 3. ストア・アイテム別の詳細分析

In [ ]:
# 3.1 ストアごとの平均売上
plt.figure(figsize=(10, 5))
sns.barplot(x='store', y='sales', data=train)
plt.title('Average Sales per Store')
plt.show()

In [ ]:
# 3.2 アイテムごとの平均売上 (上位10・下位10)
item_sales = train.groupby('item')['sales'].mean().sort_values(ascending=False)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
sns.barplot(x=item_sales.head(10).index, y=item_sales.head(10).values, ax=ax1)
ax1.set_title('Top 10 Selling Items')
sns.barplot(x=item_sales.tail(10).index, y=item_sales.tail(10).values, ax=ax2)
ax2.set_title('Bottom 10 Selling Items')
plt.show()

## 4. 自己相関分析 (ACF / PACF)

数学的に「ラグ特徴量」がなぜ重要なのかを確認します。特定のストア・アイテム（Store 1, Item 1）に注目します。

In [ ]:
sample_sales = train[(train['store'] == 1) & (train['item'] == 1)]['sales']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
sm.graphics.tsa.plot_acf(sample_sales, lags=30, ax=ax1)
sm.graphics.tsa.plot_pacf(sample_sales, lags=30, ax=ax2)
plt.suptitle('ACF & PACF for Store 1, Item 1')
plt.show()

**考察**:
- ACFが7日の倍数で山を描いている場合、強い週間周期性があることを示しています。
- 1日前、7日前、30日前などの売上が、今日の予測に強いヒントを与えることが数学的に確認できます。